In [113]:
from py2neo import Graph, ClientError
import pandas as pd
import numpy as np

---

In [3]:
#set connection variables
PORT = "7687" #database running on this port for bolt connections
USER = "neo4j" #standard user
PASSWORD = "12345678" #db password

In [4]:
#connect to database
try:
    graph = Graph('bolt://localhost:'+PORT, auth=(USER, PASSWORD))
    print('SUCCESS: Connected to the Neo4j Database.')
except Exception as e:
    print('ERROR: Could not connect to the Neo4j Database. See console for details.')
    raise SystemExit(e)

SUCCESS: Connected to the Neo4j Database.


In [5]:
def run_query(query:str):
    try:
        return graph.run(query)
    except ClientError as e:
        print(e.message)

def _reset_graph():
    q = """
        call{
        Match (n)
        detach delete n} in transactions
    """
    return run_query(q)

def _find_import_problems():
    q = """
    MATCH (n) WHERE size(labels(n)) = 0 RETURN n
    """

    return run_query(q)

---
### The first thing to do is to find/define the research communities. A community is defined by a set of keywords. Assume that the database community is defined through the following keywords: __data management, indexing, data modeling, big data, data processing, data storage and data querying__

In [14]:
DATABASE_COMMUNITY_KEYWORDS = ["data management", "indexing", "data modeling", "big data", "data processing", "data storage","data querying"]

In [28]:
"""
MATCH(researcher:Researcher)-[:AUTHOR_OF]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
WHERE kw.keyword = "data management"
RETURN researcher.name as researcher_name, kw.keyword as keyword
"""

# QUERY_COMMUNITIES = [f"""MATCH(researcher:Researcher)-[:AUTHOR_OF]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword) WHERE kw.keyword = "{kw}" RETURN researcher.name as researcher_name, kw.keyword as keyword""" 
#                      for kw in DATABASE_COMMUNITY_KEYWORDS]
# QUERY_COMMUNITIES = [f"""MATCH(researcher:Researcher)-[:AUTHOR_OF]->(:Paper)-[:MAIN_TOPIC]->(kw:Keyword) WHERE kw.keyword = "{kw}" RETURN researcher.name as researcher_name, kw.keyword as keyword""" 
#                      for kw in DATABASE_COMMUNITY_KEYWORDS]

QUERY_COMMUNITIES = [f"""MATCH(researcher:Researcher)-[:AUTHOR_OF]->(:Paper)-[:MAIN_TOPIC]->(kw:Keyword) WHERE kw.keyword = "{kw}" RETURN researcher.name as researcher_name""" 
                     for kw in DATABASE_COMMUNITY_KEYWORDS]
print("First query:")
print(QUERY_COMMUNITIES[0])

First query:
MATCH(researcher:Researcher)-[:AUTHOR_OF]->(:Paper)-[:MAIN_TOPIC]->(kw:Keyword) WHERE kw.keyword = "data management" RETURN researcher.name as researcher_name


In [33]:
communities = dict()
# communities = []

for i, query in enumerate(QUERY_COMMUNITIES):
    cursor = run_query(query)
    df = cursor.to_data_frame()

    communities[DATABASE_COMMUNITY_KEYWORDS[i]] = df
    # communities.append(df)

In [36]:
communities[DATABASE_COMMUNITY_KEYWORDS[0]]

,researcher_name
0,Kenneth V. Iserson
1,Jean-Michel Hufflen
2,Bas Luttik
3,Giacomo Bonanno
4,Maribel Fernández
...,...
2960,Celia Fernández-Aller
2961,Steven R. Kraaijeveld
2962,Matteo E. Bonfanti
2963,Andrew P. Rebera


---
### Next, we need to find the conferences and journals related to the database community (i.e., are specific to the field of databases). Assume that if 90% of the papers published in a conference/journal contain one of the keywords of the database community we consider that conference/journal as related to that community.

In [52]:
no_of_papers = run_query(f"""
                        MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
                        RETURN j.journal_name AS journal, count(p) AS no_of_papers
                        ORDER by journal
                        """)

df_no_of_papers = no_of_papers.to_data_frame()
df_no_of_papers

,journal,no_of_papers
0,Comput. Chem. Eng.,36694
1,"ETH Zurich, Department of Computer Science / T...",6
2,Hydrology and Earth System Sciences,16
3,IEEE Trans. Hum. Mach. Syst.,7030
4,J. Econ. Theory,14464
5,Sci. Eng. Ethics,11180
6,Web Intell.,1615
7,Web Intell. Agent Syst.,1768


In [55]:
no_of_papers_with_kw = run_query(f"""
                    MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
                    WHERE kw.keyword = "{DATABASE_COMMUNITY_KEYWORDS[0]}"
                    RETURN j.journal_name AS journal, count(p) AS no_of_papers_with_kw
                    ORDER by journal
                    """)

df_no_of_papers_with_kw = no_of_papers_with_kw.to_data_frame()
df_no_of_papers_with_kw

,journal,no_of_papers_with_kw
0,Comput. Chem. Eng.,226
1,Hydrology and Earth System Sciences,1
2,IEEE Trans. Hum. Mach. Syst.,49
3,J. Econ. Theory,138
4,Sci. Eng. Ethics,73
5,Web Intell.,12
6,Web Intell. Agent Syst.,13


In [67]:
JOURNAL_COMMUNITIES = df_no_of_papers.join(df_no_of_papers_with_kw.set_index("journal"), on="journal").fillna(0)
JOURNAL_COMMUNITIES["relation_rate"] = (JOURNAL_COMMUNITIES["no_of_papers_with_kw"]/JOURNAL_COMMUNITIES["no_of_papers"])
JOURNAL_COMMUNITIES

,journal,no_of_papers,no_of_papers_with_kw,relation_rate
0,Comput. Chem. Eng.,36694,226.0,0.006159
1,"ETH Zurich, Department of Computer Science / T...",6,0.0,0.000000
2,Hydrology and Earth System Sciences,16,1.0,0.062500
3,IEEE Trans. Hum. Mach. Syst.,7030,49.0,0.006970
4,J. Econ. Theory,14464,138.0,0.009541
5,Sci. Eng. Ethics,11180,73.0,0.006530
6,Web Intell.,1615,12.0,0.007430
7,Web Intell. Agent Syst.,1768,13.0,0.007353


In [93]:
# relationship between KEYWORD and CONFERENCE
f"""
MATCH(kw:Keyword)<-[r:MAIN_TOPIC]-(p:Paper)-[:PUBLISHED_IN]->(ce:ConferenceEdition)<-[holds:HOLDS]-(conf:Conference)
WHERE kw.keyword = "{DATABASE_COMMUNITY_KEYWORDS[0]}"
RETURN kw, p, ce, conf
LIMIT 2
"""

# relationship between KEYWORD and JOURNAL
f"""
MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
WHERE kw.keyword = "{DATABASE_COMMUNITY_KEYWORDS[0]}"
RETURN j, p, kw
LIMIT 2
"""

# possible solution, HIGH CPU USAGE
"""
MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
OPTIONAL MATCH (j)-[:JOURNAL_PUBLICATION]->(p2:Paper)-[:MAIN_TOPIC]->(kw2:Keyword)
WITH j, p, kw2, count(p) AS no_of_papers, count(p2) AS no_of_papers_with_kw
WHERE kw2.keyword = "data management"
RETURN j.journal_name AS name, no_of_papers, no_of_papers_with_kw
ORDER BY name
LIMIT 2
"""

QUERY_JOURNALS_WITH_KW = [f"""
                    MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
                    WHERE kw.keyword = "{kw}"
                    RETURN j.journal_name AS journal, count(p) AS no_of_papers_with_kw
                    ORDER by journal
                    """
            for kw in DATABASE_COMMUNITY_KEYWORDS]

QUERY_JOURNALS = [f"""
                    MATCH(j:Journal)-[:JOURNAL_PUBLICATION]->(p:Paper)-[:MAIN_TOPIC]->(kw:Keyword)
                    RETURN j.journal_name AS journal, count(p) AS no_of_papers
                    ORDER by journal
                    """
            for kw in DATABASE_COMMUNITY_KEYWORDS]

QUERY_CONFERENCES = [f"""
                    MATCH(kw:Keyword)<-[r:MAIN_TOPIC]-(p:Paper)-[:PUBLISHED_IN]->(ce:ConferenceEdition)<-[holds:HOLDS]-(conf:Conference)
                    RETURN conf.conference_name AS conference, count(p) AS no_of_papers
                    ORDER by conference""" 
            for kw in DATABASE_COMMUNITY_KEYWORDS]

QUERY_CONFERENCES_WITH_KW = [f"""
                    MATCH(kw:Keyword)<-[r:MAIN_TOPIC]-(p:Paper)-[:PUBLISHED_IN]->(ce:ConferenceEdition)<-[holds:HOLDS]-(conf:Conference)
                    WHERE kw.keyword = "{kw}"
                    RETURN conf.conference_name AS conference, count(p) AS no_of_papers_with_kw
                    ORDER by conference
                    """
            for kw in DATABASE_COMMUNITY_KEYWORDS]
# QUERY_2 = """

# """
# print("Second query:")
# print(QUERY_2)

In [145]:
# cursor = run_query(QUERY_2)
# cursor.to_data_frame()
# run_query(QUERY_JOURNALS_WITH_KW[0])
JOURNAL_COMMUNITIES = []
CONFERENCE_COMMUNITIES = []

for i, query in enumerate(QUERY_JOURNALS_WITH_KW):
    JOURNAL_COMMUNITY = pd.DataFrame.empty

    df_journals_with_kw = run_query(query)
    df_journals_with_kw = df_journals_with_kw.to_data_frame()
    
    df_journals = run_query(QUERY_JOURNALS[i])
    df_journals = df_journals.to_data_frame()

    JOURNAL_COMMUNITY = df_journals.join(df_journals_with_kw.set_index("journal"), on="journal").fillna(0)
    JOURNAL_COMMUNITY["relation_rate"] = (JOURNAL_COMMUNITY["no_of_papers_with_kw"]/JOURNAL_COMMUNITY["no_of_papers"])

    JOURNAL_COMMUNITIES.append(JOURNAL_COMMUNITY)
    
    # ------------- FOR CONFERENCES -------------
    CONFERENCE_COMMUNITY = pd.DataFrame.empty

    df_conferences_with_kw = run_query(QUERY_CONFERENCES_WITH_KW[i])
    df_conferences_with_kw = df_conferences_with_kw.to_data_frame()
    
    df_conferences = run_query(QUERY_CONFERENCES[i])
    df_conferences = df_conferences.to_data_frame()

    CONFERENCE_COMMUNITY = df_conferences.join(df_conferences_with_kw.set_index("conference"), on="conference").fillna(0)
    CONFERENCE_COMMUNITY["relation_rate"] = (CONFERENCE_COMMUNITY["no_of_papers_with_kw"]/CONFERENCE_COMMUNITY["no_of_papers"])

    CONFERENCE_COMMUNITIES.append(CONFERENCE_COMMUNITY)

In [146]:
print("max relation rates")
max_conf = -1
max_journal = -1

for i, e in enumerate(CONFERENCE_COMMUNITIES):
    max_conf = np.max([e["relation_rate"].max(), max_conf])
    max_journal = np.max([JOURNAL_COMMUNITIES[i]["relation_rate"].max(), max_journal])

print("max_conf:", max_conf)
print("max_journal:", max_journal)
# !!! NO COMMUNITIES, relation rates should be >90%

max relation rates
max_conf: 0.25
max_journal: 0.16666666666666666


In [147]:
# example COMMUNITY
CONFERENCE_COMMUNITIES[0]

,conference,no_of_papers,no_of_papers_with_kw,relation_rate
0,ABMB,69,1.0,0.014493
1,ACAC,4,0.0,0.000000
2,ACCAT@ETAPS,28,0.0,0.000000
3,ACL2,103,1.0,0.009709
4,ACML,513,5.0,0.009747
...,...,...,...,...
423,Workshop on Type-Theoretic Languages: Proof Se...,40,0.0,0.000000
424,YR-SOC,26,0.0,0.000000
425,Yahoo! Learning to Rank Challenge,39,0.0,0.000000
426,ateM,94,2.0,0.021277


In [161]:
# for EXAMPLE COMMUNITY
# CONFERENCE_COMMUNITIES[0][CONFERENCE_COMMUNITIES[0]["relation_rate"] > 0.9]

# for all COMMUNITIES, i.e. >0.9 relation_rate
# df: pd.DataFrame
for i, df in enumerate(DATABASE_COMMUNITY_KEYWORDS):
    # CONFERENCE_COMMUNITIES[i] = CONFERENCE_COMMUNITIES[i][CONFERENCE_COMMUNITIES[i]["relation_rate"] > 0.9]
    # JOURNAL_COMMUNITIES[i] = JOURNAL_COMMUNITIES[i][JOURNAL_COMMUNITIES[i]["relation_rate"] > 0.9]
    CONFERENCE_COMMUNITIES[i] = CONFERENCE_COMMUNITIES[i][CONFERENCE_COMMUNITIES[i]["relation_rate"] > 0.1]
    JOURNAL_COMMUNITIES[i] = JOURNAL_COMMUNITIES[i][JOURNAL_COMMUNITIES[i]["relation_rate"] > 0.1]
    # print(df, "\n-----------------")

In [165]:
CONFERENCE_COMMUNITIES[0]

,conference,no_of_papers,no_of_papers_with_kw,relation_rate
28,BEAT,5,1.0,0.200
152,HCVS/VPT@ETAPS,5,1.0,0.200
271,PrePost@IFM,8,1.0,0.125


In [179]:
# add communities

COMMUNITY_QUERIES = []

for i, kw in enumerate(DATABASE_COMMUNITY_KEYWORDS):
    conferences = CONFERENCE_COMMUNITIES[i]["conference"]
    for j, conference in enumerate(conferences):
        QUERY_ADD_CONFERENCE_COMMUNITIES = """
        CALL {
            MERGE (c:Community {related_keyword:"%s"})
            ON CREATE
                SET c.conference = "%s"
            ON MATCH
                SET c.conference = "%s"
        } IN TRANSACTIONS
        """ % (kw, conference, conference)

        QUERY_ADD_CONFERENCE_COMMUNITIES_RELATIONSHIPS = """
        MATCH (community:Community {related_keyword:"%s"}), (conference:Conference {conference_name:"%s"})
        MERGE (conference)-[:RELATED_TO]->(community)
        """ % (kw, conference)

        COMMUNITY_QUERIES.append(QUERY_ADD_CONFERENCE_COMMUNITIES)
        COMMUNITY_QUERIES.append(QUERY_ADD_CONFERENCE_COMMUNITIES_RELATIONSHIPS)

        print(QUERY_ADD_CONFERENCE_COMMUNITIES)
        break

    # -------- for JOURNAL --------
    journals = JOURNAL_COMMUNITIES[i]["journal"]
    for j, journal in enumerate(journals):

        QUERY_ADD_JOURNAL_COMMUNITIES = """
        CALL {
            MERGE (c:Community {related_keyword:"%s"})
            ON CREATE
                SET c.journal = "%s"
            ON MATCH
                SET c.journal = "%s"
        } IN TRANSACTIONS
        """ % (kw, journal, journal)

        QUERY_ADD_JOURNAL_COMMUNITIES_RELATIONSHIPS = """
        MATCH (community:Community {related_keyword:"%s"}), (journal:Journal {journal_name:"%s"})
        MERGE (journal_name)-[:RELATED_TO]->(community)
        """ % (kw, journal)

        COMMUNITY_QUERIES.append(QUERY_ADD_JOURNAL_COMMUNITIES)
        COMMUNITY_QUERIES.append(QUERY_ADD_JOURNAL_COMMUNITIES_RELATIONSHIPS)
        
        # break

    # run_query(QUERY_ADD_CONFERENCE_COMMUNITIES)
    # run_query(QUERY_ADD_CONFERENCE_COMMUNITIES_RELATIONSHIPS)
    # run_query(QUERY_ADD_JOURNAL_COMMUNITIES)
    # run_query(QUERY_ADD_JOURNAL_COMMUNITIES_RELATIONSHIPS)

    break


        CALL {
            MERGE (c:Community {related_keyword:"data management"})
            ON CREATE
                SET c.conference = "BEAT"
            ON MATCH
                SET c.conference = "BEAT"
        } IN TRANSACTIONS
        


In [180]:
# run the QUERIES

# for _, query in enumerate(COMMUNITY_QUERIES):
#     run_query(query)

---
### Next, we want to identify the top papers of these conferences/journals. We need to find the papers with the highest page rank provided the number of citations from the papers of the same community (papers in the conferences/journals of the database community). As a result we would obtain (highlight), say, the top-100 papers of the conferences of the database community.
